In [1]:
! find . -name "*.pyc" -exec rm -f {} \;

In [2]:
import warnings

def fxn():
    warnings.warn("UserWarning", UserWarning)

#with warnings.catch_warnings():
warnings.simplefilter("default")
#fxn()

In [3]:
from regr.graph import Graph
from regr.entity import Entity
from regr.relation import Relation

In [4]:
with Graph('global') as graph:
    with Graph('linguistic') as ling_graph:
        # linguistic layer
        word = Entity(name='word')
        phrase = Entity(name='phrase')
        sentence = Entity(name='sentence')
        phrase.have(word)
        sentence.have(phrase)

    with Graph('application') as app_graph:
        # application nodes
        entity = Entity(name='entity')
        entity.be(phrase)
        pair = Entity(name='pair')
        pair.be((entity, entity))

        people = Entity(name='people')
        organization = Entity(name='organization')
        other = Entity(name='other')
        people.be(entity)
        organization.be(entity)
        other.be(entity)

        work_for = Entity(name='work-for')
        work_for.be({'employee':people, 'employer':organization})
        work_for.be(pair)

graph

# some other sample of planed graph API, hopefully make sense towards turing completeness
#
# two.be((people, people))
# colleague.be(two)
# friend.be(two)
# meet.be(two)
# beer.be(two)
# argue.be(two)
# (colleague or friend).be(meet)
# (colleague and friend).be(beer)
# (colleague and not friend).be(argue)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, {'employee': Entity(name='people', what={'rels': {Entity(name='entity'): set([Be(name='(people)-be-(0:entity)', what={'dst': OrderedDict([(0, Entity(name='entity'))]),
 'src': Entity(name='people')})])}}), 'employer': Entity(name='organization', what={'rels': {Entity(name='entity'): set([Be(name='(organization)-be-(0:entity)', what={'dst': OrderedDict([(0, Entity(name='entity'))]),
 'src': Entity(name='organization')})])}})} is used.
regr/relation.py:11: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, {'employee': Entity(name='people', what={'rels': {Entity(name='entity'): set([Be(name='(people)-be-(0:entity)', what={'dst': OrderedDict([(0, Entity(name='entity'))]),
 'src': Entity(name='people')})])}}), 'employer': Entity(name='organization'

Graph(name='global', what={'entities': [],
 'subgraphs': [Graph(name='linguistic', what={'entities': [Entity(name='word', what={'rels': {}}),
              Entity(name='phrase', what={'rels': {Entity(name='word'): set([Have(name='(phrase)-have-(0:word)', what={'dst': OrderedDict([(0, Entity(name='word'))]),
 'src': Entity(name='phrase')})])}}),
              Entity(name='sentence', what={'rels': {Entity(name='phrase'): set([Have(name='(sentence)-have-(0:phrase)', what={'dst': OrderedDict([(0, Entity(name='phrase'))]),
 'src': Entity(name='sentence')})])}})],
 'subgraphs': [],
 'supergraph': Graph(name='global')}),
               Graph(name='application', what={'entities': [Entity(name='entity', what={'rels': {Entity(name='phrase'): set([Be(name='(entity)-be-(0:phrase)', what={'dst': OrderedDict([(0, Entity(name='phrase'))]),
 'src': Entity(name='entity')})])}}),
              Entity(name='pair', what={'rels': {Entity(name='entity'): set([Be(name='(pair)-be-(0:entity,1:entity)', what={'

In [5]:
print Graph._names
print Entity._names
print Relation._names

Counter({'linguistic': 1, 'global': 1, 'application': 1})
Counter({'word': 1, 'people': 1, 'sentence': 1, 'work-for': 1, 'entity': 1, 'other': 1, 'organization': 1, 'pair': 1, 'phrase': 1})
Counter({'(pair)-be-(0:entity,1:entity)': 1, '(work-for)-be-(0:pair)': 1, '(other)-be-(0:entity)': 1, '(phrase)-have-(0:word)': 1, '(sentence)-have-(0:phrase)': 1, '(people)-be-(0:entity)': 1, '(organization)-be-(0:entity)': 1, '(entity)-be-(0:phrase)': 1, '(work-for)-be-(employee:people,employer:organization)': 1})


In [6]:
# pytorch something
# numpy for example

import numpy as np
from regr.utils import extract_args


def np_debug(mat, show_value=False):
    args = extract_args(_stack_back_level_=1)
    for k, v in args.items():
        v = np.array(v)
        print('{}: {}, {}'.format(k, v.shape, v.dtype))
        if show_value:
            print(v)


def mask_expand(mask_len, max_len):
    mask_len = np.array(mask_len)
    expanded = np.tile(np.expand_dims(
        np.arange(max_len), axis=0), mask_len.shape + (1,))
    return expanded < np.expand_dims(mask_len, axis=-1)


def softmax(x): return np.exp(x) / np.exp(x).sum(axis=-1, keepdims=True)


# config and data
batch_size = 2
max_len = 10
max_phrase = 5
mask_len = np.random.randint(max_len / 4, max_len, size=batch_size)
np_debug(mask_len)
repr_size = 16

entity_types = 3
relation_types = 2

mask = mask_expand(mask_len, max_len)
np_debug(mask)  # (batch, word,)

# belief layer
bword = np.ones((batch_size, max_len)) * mask
np_debug(bword)  # (batch, word,)
# embedding layer
rword = np.random.random((batch_size, max_len, repr_size)) * \
    np.expand_dims(mask, axis=-1)  # (batch, batch, repr)
np_debug(rword)

# links
# phrase -.-> word
tokenizor = np.random.random(
    (batch_size, max_phrase, max_len)) * np.expand_dims(mask, axis=1)
tokenizor = (tokenizor == tokenizor.max(axis=1, keepdims=True)
             ) & np.expand_dims(mask, axis=1)
np_debug(tokenizor)  # (batch, prhase, word,)

phrase_mask = tokenizor.sum(axis=-1).astype(np.bool)
np_debug(phrase_mask)  # (batch, prhase,)

# sentence -.-> phrase
sentence_rel = np.ones((batch_size, max_phrase), dtype=np.bool) * phrase_mask
np_debug(sentence_rel)  # (batch, prhase,)

# entity ---> phrase
entity_pred = np.random.choice(
    [False, True], (batch_size, max_phrase)) * phrase_mask
np_debug(entity_pred)  # (batch, prhase,)

# people|organization|other ---> entity
entity_clas = np.random.randn(batch_size, max_phrase, entity_types)
entity_clas = softmax(entity_clas) * np.expand_dims(phrase_mask, axis=-1)
np_debug(entity_clas)  # (batch, prhase, entity_type,)

relation_mask = np.matmul(np.expand_dims(
    phrase_mask, axis=2), np.expand_dims(phrase_mask, axis=1))
np_debug(relation_mask)  # (batch, prhase, prhase,)

# work_for ---> pair
relation_clas = np.random.randn(
    batch_size, max_phrase, max_phrase, relation_types)
relation_clas = softmax(relation_clas) * np.expand_dims(relation_mask, axis=-1)
np_debug(relation_clas)  # (batch, prhase, prhase, relation_type,)


mask_len: (2,), int64
mask: (2, 10), bool
bword: (2, 10), float64
rword: (2, 10, 16), float64
tokenizor: (2, 5, 10), bool
phrase_mask: (2, 5), bool
sentence_rel: (2, 5), bool
entity_pred: (2, 5), bool
entity_clas: (2, 5, 3), float64
relation_mask: (2, 5, 5), bool
relation_clas: (2, 5, 5, 2), float64


In [7]:
entity['confidence', 1.0] = entity_gt
entity['confidence'] = entity_pred

word['feature'] = rword

JJ['confidence'] = lr(w2c(word['feature']))[:,:,0]
NN['confidence'] = lr(w2c(word['feature']))[:,:,1]

work_for['confidence'] = isworkfor(pair['feature'])

NameError: name 'entity_gt' is not defined